# ENSO Extratropical Prediction Project

## Problem Description
The scientific goal of this project is to investigate the impact of the extratropics on ENSO predictability and prediction at 1-year lead times. Previous work of mine (add references) as well as many others (add references), have demontrated a link between the extratropics and ENSO development. Many features (predictors) have been identified and they are highly correlated with each other.  Can we identify a better feature or set of features by learning from all of the data rather than a priori choosing features? Can we use this to help us understand the physical and dynamical processes by which the extratropics impact development of ENSO?

### What do we want to predict?

We want to predict Eastern Pacific or Central Pacific ENSO Events in Dec-Jan-Feb (DJF1) from the previous Dec-Jan-Feb (DJF0).  This is defined by indices, referred to as CP for Central Pacific or EP for Eastern Pacific. 

As a regression problem, we would just predict the value of the CP or EP Index in DJF1

As a classification problem, we would define the predictand as:
If the index is >=0.5 for DJF1, then we have a CP or EP El Nino (warm event)
If the index is <=-0.5 for DJF1, then we have a CP or EP La Nina (cold event)

For the purposes of this notebook, will write out the value of the CP or EP index to allow for flexibility in how we want to formulate the problem.

### How will we predict it?

Features (predictors)
:The features for this problem are detrended anomalies of Sea Surface Temperature (SST), sea level pressure (SLP), zonal wind stress (TAUX), zonal wind at 10m (UWND), and meridional wind at 10m (VWND) from observations based datasets at every gridpoint in the Pacific region (65S-85N ;100E-70W). SST and TAUX fields are defined only over the ocean with land points set as np.nan.  SLP, UWND, and VWND are defined over the entire domain (land and ocean).  The correlation with the CP and EP ENSO indices has been removed via linear regression.The features have not been normalized and will likely need to be because the have different scales and variance.

Predictand
: Normalized values of the CP and EP ENSO indices

### Dataset Reference
1. SST is from NOAA/ERSSTv3b (1x1) 181x89
2. SLP from NCEP Reanalysis 1 (2.5x2.5) 73x144
3. TAUX from NCEP Reanalysis 1 Gaussian Grid 94x192
4. UWND from NCEP Reanalysis 1 Gaussian Grid 94x192
5. VWND from NCEP Reanalysis 1 Gaussian Grid 94x192

All data are interpolated to the 2.5x2.5 deg grid

## This notebook will:
1. read in the features and predictands from netcdf as monthly data
2. Extract the Pacific from the features datasets
3. Calculate and extract the DJF0 seasonal averages for the features and the DJF1 seasonal averages from the predictand
4. convert the data to a Pandas DataFrame
5.  write out as a .csv file

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe
import matplotlib.pyplot as plt

Read in the file that has the common grid we will interpolate other fields to

In [2]:
gridPath='/project/predictability/kpegion/rmenso/data/obs/'
gridFile='NCEPR1.slp.anom.detrend.SEAS.globalreg.1948-2014.nc'
ds_grid=xr.open_dataset(gridPath+gridFile, decode_times=False)
ds_out = xr.Dataset({'lat': ds_grid['lat'],
                     'lon': ds_grid['lon'],})

Define a list of the variables 

In [3]:
varnames=['slp','uflx','uwnd','vwnd','sst']

Setup path and filenames for the input data

In [4]:
dataPath2='/project/predictability/kpegion/rmenso/data/obs/'
fnames=['NCEPR1.slp.anom.detrend.SEAS.globalreg.1948-2014.nc',
        'NCEPR1.uflx.anom.detrend.SEAS.globalreg.1948-2014.nc',
        'NCEPR1.uwnd.anom.detrend.SEAS.globalreg.1948-2014.nc',
        'NCEPR1.vwnd.anom.detrend.SEAS.globalreg.1948-2014.nc',
        'ERSST.sst.anom.detrend.SEAS.globalreg.1948-2014.nc']

Read in the data files for the predictors (features), interpolate to common grid, create a list of xarray.Datasets

In [ ]:
# Create empty list for storing xarray.Datasets
ds_list=[]

# Loop over all the variables and files
for f,v in zip(fnames,varnames):

    # Read in the data
    dsF=xr.open_dataset(dataPath2+f, decode_times=False)
    
    # Rename the DataArray to the variable name
    dsF=dsF.rename({'DJF':v})
    
    # Regrid the data to the common grid
    regridder = xe.Regridder(dsF, ds_out, 'bilinear')
    ds_out = regridder(dsF)  
    
    # Set land missing values to NaN for SST
    if (v=='sst'):
        ds_out=ds_out.where(ds_out['sst']<9999,np.nan)
    
    # Append this xarray.Dataset to the list
    ds_list.append(ds_out[v])

# Clean up regridding weight file
regridder.clean_weight_file() 

Overwrite existing file: bilinear_73x144_73x144.nc 
 You can set reuse_weights=True to save computing time.


Define the path and file for the predictands

In [ ]:
dataPath1='/project/predictability/kpegion/rmenso/data/analysis/enso/'
fnamecp='cp.pc.ERSST.SEAS.nc'
fnameep='ep.pc.ERSST.SEAS.nc'

Read in the predictands and append to the xarray.Dataset list

In [ ]:
ds_cp=xr.open_dataset(dataPath1+fnamecp, decode_times=False).squeeze()
ds_ep=xr.open_dataset(dataPath1+fnameep, decode_times=False).squeeze()

ds_cp=ds_cp.rename({'djf1':'cp'})
ds_ep=ds_ep.rename({'djf1':'ep'})

ds_list.append(ds_cp['cp'])
ds_list.append(ds_ep['ep'])

Combine all the xarray.Dataset variables into a single xarray.Dataset and assign the date/times properly

In [ ]:
ds=xr.merge(ds_list)   
ds

Extract the Pacific Ocean region from 65S to 85N and 100E to 70W

In [ ]:
ds_pac=ds.sel(lat=slice(-65,85),lon=slice(100,290))
ds_pac

Quick check plot

In [ ]:
plt.contourf(ds_pac['sst'][0,:,:])
plt.colorbar()

Convert to pandas dataframe

In [ ]:
pd=ds_pac.to_dataframe().round(5)
pd

Write to csv

In [ ]:
pd.to_csv('/scratch/kpegion/pacocnDJFcpepDJF1.csv')